In [1]:
from IPython.display import clear_output

def load_from_gdrive(gid, filename='temp'):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$gid -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$gid -O $filename && rm -rf /tmp/cookies.txt

In [2]:
!rm -rf data

data_id = '1MsS9p8Zz-Kee1Yi-ZhYOc8tHGH1dCOLK'
load_from_gdrive(data_id, 'data.zip')
!unzip -P Skoltech data.zip -d /content/data/

clear_output()

In [3]:
!pip install -U pyarrow implicit
clear_output()

# Data

Loading and preprocessing data

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DATA_DIR = '/content/data'
CLIENTS_FILE = os.path.join(DATA_DIR, 'clients.csv')
MATERIALS_FILE = os.path.join(DATA_DIR, 'materials.csv')
PLANTS_FILE = os.path.join(DATA_DIR, 'plants.csv')
TRANSACTIONS_FILE = os.path.join(DATA_DIR, 'transactions.parquet')

In [2]:
df_clients = pd.read_csv(CLIENTS_FILE)
print(df_clients.shape)
df_clients.head()

(99995, 4)


,client_id,gender,city,birthyear
0,1a47d62dddacc03fe90c15652f7ae1a4,M,Other,1990.0
1,9cb909f701d25d548e953bff81192b56,F,Other,1969.0
2,d5da0f5b841b4f83383202807995027a,M,Other,1976.0
3,13ed7f16810b17b8cee6de834ac79a48,F,Moscow,1966.0
4,215fe3ea7d5bf0415e5504e2a7f33551,F,Other,1988.0


In [3]:
df_materials = pd.read_csv(MATERIALS_FILE)
print(df_materials.shape)
df_materials.head()

(105609, 8)


,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,35cbdf61de9e19c8b417327aaef14c88,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,9eed45f71360b4b1e2590637467220e5,212a38db0ddcd009f1e164cc8483485c,0,0
1,c0b0bf24d4ec71da3d304f761ec555d8,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,de2eb747e0896c050905a6b635ab800a,9b0b6c7d55413ad3b67761b7b125b534,0,0
2,f0fc5e654a81a7c4b8ba8d7c26546e14,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,8a969031832c535daf96e0c2aed8e814,9b0b6c7d55413ad3b67761b7b125b534,0,0
3,353693e64fb5f9e2d29746d7fe6edf1e,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
4,5d9e0b4302ce95448cae72165ff4cf5b,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0


In [4]:
df_plants = pd.read_csv(PLANTS_FILE)
print(df_plants.shape)
df_plants.head()

(388, 3)


,plant,plant_type,city
0,95b09698fda1f64af16708ffb859eab9,HM,St. Petersburg
1,926abae84a4bd33c834bc6b981b8cf30,HM,St. Petersburg
2,ae2bac2e4b4da805d01b2952d7e35ba4,HM,St. Petersburg
3,0e7e3cf0ded4d9db8b376b317c007f99,HM,St. Petersburg
4,540bd55a2cf295b8ea9cd78650e89d03,HM,St. Petersburg


In [5]:
df_transactions = pd.read_parquet(TRANSACTIONS_FILE, engine='pyarrow', use_threads=True)
print(df_transactions.shape)
df_transactions.head()

(32109414, 9)


,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
0,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,004,90fca68123e692a005a4edeadd94b2f2,232cfe8b28970434f685b737737b26d4,2.0,146.98,0
1,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,003,90fca68123e692a005a4edeadd94b2f2,12ec64451b8e426db16f3f05ea770901,1.0,249.99,1
2,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,012,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1
3,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,013,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1
4,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,011,90fca68123e692a005a4edeadd94b2f2,571c6d3b559db8445e05265649eb30ab,2.0,53.98,1


Clients and materials id mappings:

In [6]:
clients_map = {client_id: i for i, client_id in enumerate(df_clients.client_id.values)}
materials_map = {material: i for i, material in enumerate(df_materials.material.values)}

materials_fields_maps = {}
for col in ['hier_level_1', 'hier_level_2', 'hier_level_3', 'hier_level_4', 'vendor']:
    val2id = {
        value: i for i, value in enumerate(df_materials[col].unique())
    }
    materials_fields_maps[col] = {
        materials_map[row[0]]: val2id[row[1]] for row in df_materials[['material', col]].values
    }
    materials_fields_maps[col + '_ids'] = val2id
    materials_fields_maps[col + '_ids_inv'] = {x[1]: x[0] for x in val2id.items()}

Adding new fields to the transactions table + filtering some rows

In [7]:
# remove returns
df_transactions = df_transactions[df_transactions.sales_count > 0.]
print(df_transactions.shape)

# filter transactions by available client and material data
df_transactions = df_transactions[df_transactions.client_id.isin(clients_map)]
print(df_transactions.shape)

df_transactions = df_transactions[df_transactions.material.isin(materials_map)]
print(df_transactions.shape)

# merge with materials
# df_transactions = df_transactions.merge(df_materials)
# df_transactions.sort_values(by=['chq_date'], inplace=True)

# add week information
df_transactions['week'] = df_transactions['chq_date'].apply(lambda x: x.isocalendar()[1])

# add product price
# df_transactions['material_price'] = df_transactions.sales_sum / df_transactions.sales_count

(32086641, 9)
(32085025, 9)
(32072077, 9)


In [8]:
df_transactions.head()

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo,week
0,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,004,90fca68123e692a005a4edeadd94b2f2,232cfe8b28970434f685b737737b26d4,2.0,146.98,0,44
1,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,003,90fca68123e692a005a4edeadd94b2f2,12ec64451b8e426db16f3f05ea770901,1.0,249.99,1,44
2,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,012,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1,44
3,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,013,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1,44
4,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,011,90fca68123e692a005a4edeadd94b2f2,571c6d3b559db8445e05265649eb30ab,2.0,53.98,1,44


# Recommendation models

Collaborative filtering using Alternating Least Squares and `implicit` package.

The models are built from the transaction data for each client with the following levels of item granularity:

- `material`s (products)
- `hier_level_2` categories
- `hier_level_3` categories
- `hier_level_4` categories
- `vendor`s

Using multiple models we can offer recommendations with various levels of product description: from raw product to categories. This approach allows to build personalized tasks based on categories, vendors or products and product descriptions themselves.

## Model fit

In [9]:
from scipy.sparse import coo_matrix
import implicit

def fit_als_model(field, alpha_val=15, factors=64, regularization=0.1, iterations=50, random_state=0):
    if field == 'material':
        items = materials
    else:
        items = materials.apply(lambda x: materials_fields_maps[field][x])

    user_item = coo_matrix((np.ones_like(clients), (clients, items))).tocsr()
    item_user = user_item.T

    model = implicit.als.AlternatingLeastSquares(
        factors=factors, regularization=regularization, iterations=iterations,
        calculate_training_loss=True, random_state=random_state
    )

    data_conf = (item_user * alpha_val).astype('double')
    model.fit(data_conf)

    return model, user_item

In [10]:
clients = df_transactions.client_id
materials = df_transactions.material

# map to integer ids
clients = clients.apply(lambda x: clients_map[x])
materials = materials.apply(lambda x: materials_map[x])

In [11]:
model_material, user_item_material = fit_als_model(field='material', factors=64)
model_hier_2, user_item_hier_2 = fit_als_model(field='hier_level_2', alpha_val=1., factors=32)
model_hier_3, user_item_hier_3 = fit_als_model(field='hier_level_3', alpha_val=1., factors=32)
model_hier_4, user_item_hier_4 = fit_als_model(field='hier_level_4', alpha_val=1., factors=32)
model_vendor, user_item_vendor = fit_als_model(field='vendor', alpha_val=1., factors=32)

## Recommendation function

The whole power of multiple recommendation models and customization in one function:

In [12]:
def recommend_items(model, user_item, client_id, field='material', N=10, top_k=None, 
                        mode='all', only_positive=True, **filters):
    """Recommend items based on the conditions.
    
    Args:
        model: ALS model
        user_item: sparse user-item matrix
        client_id: integer id of the client for whom recomendations should be computed
        field: model and recommendations levels, possible options: 
            `material`, `hier_level_2`, `hier_level_3`, `hier_level_4`, `vendor`
        N: how many (maximum) items to recommend
        top_k: (optional) how many items to consider in the whole set of items
            (to speed up computations)
        mode: operation mode, possible options: 
            `all` -- all items (including bought in the past)
            `bought` -- consider only bought items
            `novel` -- only new items
        only_positive: consider only items with positive recomendation scores
        **filters: optional `field=value` filters for `materials`
    """
    result = []

    ranked_items = model.rank_items(0, user_item, np.arange(user_item.shape[1]))
    top_k = len(ranked_items) if top_k is None else top_k

    for i, (item_id, score) in enumerate(ranked_items[:top_k]):
        # get material purchase frequency from the transactions
        frequency = user_item.getrow(client_id).getcol(item_id).toarray()[0][0]

        # filter by operation mode
        if mode == 'bought' and frequency == 0:
            continue
        elif mode == 'novel' and frequency > 0:
            continue
        elif only_positive and score < 0.:
            break

        if field == 'material':
            item = df_materials.iloc[item_id]

            # filter by provided fields
            skip_item = False
            for key, value in filters.items():
                if key in item and item[key] != value:
                    skip_item = True
                    break
            if skip_item:
                continue
        else:
            item = materials_fields_maps[field + '_ids_inv'][item_id]

        # all good, save recommended item
        result.append({
            'overall_rank': i,
            'item_id': item_id,
            'item': item,
            'score': score,
            'frequency': frequency,
        })

        if len(result) == N:
            break

    return result

Testing the function:

In [13]:
recommend_items(
    model_material, user_item_material, client_id=0, field='material',
    N=3, top_k=10000,
    mode='bought', # 'all', 'bought', 'novel'
    # is_private_label=True,
    # is_alco=True,
    hier_level_1='FOOD',
    # hier_level_2='33e75ff09dd601bbe69f351039152189',
    # hier_level_4='29a90297dcc11ab043a1ffb971246c08'
)

[{'frequency': 2, 'item': material            a2ddcb2c3c8c4eb128286e8e552b4996
  hier_level_1                                    FOOD
  hier_level_2        33e75ff09dd601bbe69f351039152189
  hier_level_3        a3788c8c64fd65c470e23e7534c3ebc8
  hier_level_4        29a90297dcc11ab043a1ffb971246c08
  vendor              fff50291dc19f2ba2c6ad287a41f90b1
  is_private_label                                   0
  is_alco                                            1
  Name: 11175, dtype: object, 'item_id': 11175, 'overall_rank': 1, 'score': 1.1545928},
 {'frequency': 1, 'item': material            8f4b2b5d481140b01c5b20bbf493b2b9
  hier_level_1                                    FOOD
  hier_level_2        c16a5320fa475530d9583c34fd356ef5
  hier_level_3        62f91ce9b820a491ee78c108636db089
  hier_level_4        508648865001185b6a704e553c44b0d7
  vendor              af2302acfdaa5fa4a65222265856f7af
  is_private_label                                   0
  is_alco                              

In [14]:
recommend_items(
    model_hier_2, user_item_hier_2, client_id=0, field='hier_level_2',
    N=5, mode='novel', # 'all', 'bought', 'novel',
)

[{'frequency': 0,
  'item': '70efdf2ec9b086079795c442636b55fb',
  'item_id': 6,
  'overall_rank': 16,
  'score': 0.99013186},
 {'frequency': 0,
  'item': '6364d3f0f495b6ab9dcf8d3b5c6e0b01',
  'item_id': 36,
  'overall_rank': 26,
  'score': 0.9742024},
 {'frequency': 0,
  'item': '3c59dc048e8850243be8079a5c74d079',
  'item_id': 52,
  'overall_rank': 34,
  'score': 0.15559483},
 {'frequency': 0,
  'item': '03afdbd66e7929b125f8597834fa83a4',
  'item_id': 2,
  'overall_rank': 35,
  'score': 0.13063186},
 {'frequency': 0,
  'item': '1f0e3dad99908345f7439f8ffabdffc4',
  'item_id': 56,
  'overall_rank': 36,
  'score': 0.10043204}]

In [15]:
recommend_items(
    model_vendor, user_item_vendor, client_id=0, field='vendor',
    N=5, mode='all', # 'all', 'bought', 'novel',
)

[{'frequency': 4,
  'item': '630ff52a66ba2631c24227d775580871',
  'item_id': 541,
  'overall_rank': 0,
  'score': 1.0886554},
 {'frequency': 5,
  'item': 'a7a236a9bc4cfba23a7895109c6333b0',
  'item_id': 1585,
  'overall_rank': 1,
  'score': 1.0589583},
 {'frequency': 6,
  'item': 'a3d2de7675556553a5f08e4c88d2c228',
  'item_id': 63,
  'overall_rank': 2,
  'score': 1.0441481},
 {'frequency': 12,
  'item': '555b60081984bd59d637c5f6b4aa3bdb',
  'item_id': 1582,
  'overall_rank': 3,
  'score': 1.0360183},
 {'frequency': 3,
  'item': '3e9e3d22f8052f4eb926f2f82c226ed2',
  'item_id': 542,
  'overall_rank': 4,
  'score': 1.0155807}]

# Tasks

Examples of tasks with provided task generation and check functions.

In [36]:
# specify a client
client_idx = np.random.randint(0, len(clients_map)) # 15712
client = df_clients.iloc[client_idx]
client_id = client.client_id
print(client)

client_id    d951b9ab6386d315e43a3740751d7f31
gender                                      F
city                                    Other
birthyear                                1964
Name: 65421, dtype: object


In [17]:
# get last client check for demonstration purposes
def get_last_check(client_id):
    client_transactions = df_transactions[df_transactions.client_id == client_id].sort_values('chq_date')
    chq_id = client_transactions.iloc[-1].chq_id
    check = client_transactions[client_transactions.chq_id == chq_id]

    return check

###***Buy and rate***

Goal: Recommending known and previously bought products to rate them in the app

In [18]:
def get_buy_and_rate_task(client_id, mode='bought'):
    print(f'*Buy and rate ({mode} product)*\n')

    material_names = recommend_items(
        model_material, user_item_material, client_id=client_id, field='material',
        N=1, top_k=10000,
        mode=mode, # 'all', 'bought', 'novel'
    )

    mat_list = list(map(lambda x: x['material'], list(map(lambda x: x['item'], material_names))))
    material = mat_list[0]
    print(f'Buy {material} and rate it on our app')

    return material

def check_buy_and_rate_task(client_id, material_task):
    check = get_last_check(client_id)
    materials_bought = set(check.material.unique())
    # + checking if material was rated
    return material_task in materials_bought

#### *Buy and rate (known materials)*

Task: Buy a `proposed/recommended` material and rate it on our app.

Implementation: Suggest "bought" with high recommendation score

In [19]:
material_brk = get_buy_and_rate_task(client_idx)

*Buy and rate (bought product)*

Buy 1eb4a5458f072b09012fcc10f2a6f86f and rate it on our app


Check task completion based on the last receipt:

In [20]:
success_brk = check_buy_and_rate_task(client_id, material_brk)
print(f'Task completed: {success_brk}')

Task completed: True


#### *Buy and rate (novel materials)*

Task: Buy a `proposed/recommended` material and rate it on our app.

Implementation: Suggest "novel" with high recommendation score

In [21]:
material_brn = get_buy_and_rate_task(client_idx, mode='novel')

*Buy and rate (novel product)*

Buy 604de26b1dc3282336e925e9ab50d184 and rate it on our app


Check task completion based on the last receipt.

Note: no success since we suggest to buy products in novel categories

In [22]:
success_brn = check_buy_and_rate_task(client_id, material_brn)
print(f'Task completed: {success_brn}')

Task completed: False


###***Fan of Lenta's goods*** 

Task: Buy `proposed/recommended` Lenta's products.

Implementation: Recommend few Lenta's goods.

In [23]:
def get_fan_of_lenta_task(client_id, num_materials=3, mode='bought'):
    print(f"*Fan of Lenta's goods ({mode} products)*")

    material_names = recommend_items(
        model_material, user_item_material, client_id=client_id, field='material',
        N=num_materials, top_k=10000,
        mode=mode, # 'all', 'bought', 'novel'
        is_private_label=1
    )

    materials = list(map(lambda x: x['material'], list(map(lambda x: x['item'], material_names))))
    print("Buy at least one of the following Lenta's products:\n")
    print('Example products:')
    for m in materials:
        print(f' - {m}')
    return materials

def check_fan_of_lenta_task(client_id, materials_task):
    check = get_last_check(client_id)
    materials_bought = set(check.material.unique())
    materials_task = set(materials_task)
    materials_intersection = materials_task.intersection(materials_bought)
    return len(materials_intersection) >= 1, materials_intersection

In [24]:
materials_fl = get_fan_of_lenta_task(client_idx, num_materials=3, mode='novel')

*Fan of Lenta's goods (novel products)*
Buy at least one of the following Lenta's products:

Example products:
 - 604de26b1dc3282336e925e9ab50d184
 - 2ecb95feadeda76e0198f0ad0c3b991d
 - b75147feae4cf8ec07fb5f2cef855def


Check task completion based on the last receipt:

In [25]:
success_fl, materials_fl_success = check_fan_of_lenta_task(
    client_id, materials_fl
)
print(f'Task completed: {success_fl}')
print(f'Bought {len(materials_fl_success)} asked Lenta products:\n {materials_fl_success}')

Task completed: False
Bought 0 asked Lenta products:
 set()


###***New vendor***

Task: Buy goods of `proposed/recommended` vendors.

Implementation: Recommend few vendors.

In [26]:
def get_new_vendor_task(client_id, num_vendors=3, mode='novel'):
    print(f"*New vendor ({mode} products)*")

    vendor_names = recommend_items(
        model_vendor, user_item_vendor, client_id=client_id, field='vendor',
        N=num_vendors, mode=mode # 'all', 'bought', 'novel',
    )
    vendors = list(map(lambda x: x['item'], vendor_names))
    print("Buy goods at least of the one of the following vendors:\n")
    print('Example vendors:')
    for m in  vendors:
        print(f' - {m}')
    return vendors

def check_new_vendor_task(client_id, vendors_task):
    check = get_last_check(client_id)
    vendors_bought = check.material\
        .apply(lambda x: materials_map[x])\
        .apply(lambda x: materials_fields_maps['vendor'][x])\
        .apply(lambda x: materials_fields_maps['vendor_ids_inv'][x]).unique()
    vendors_bought = set(vendors_bought)
    vendors_task = set(vendors_task)
    vendors_intersection = vendors_task.intersection(vendors_bought)
    return len(vendors_intersection) >= 1, vendors_intersection

In [27]:
vendors_nv = get_new_vendor_task(client_idx, num_vendors=3, mode='novel')

*New vendor (novel products)*
Buy goods at least of the one of the following vendors:

Example vendors:
 - 630ff52a66ba2631c24227d775580871
 - a7a236a9bc4cfba23a7895109c6333b0
 - a3d2de7675556553a5f08e4c88d2c228


Check task completion based on the last receipt:

In [28]:
success_nv, vendors_nv_success = check_new_vendor_task(
    client_id, vendors_nv
)
print(f'Task completed: {success_nv}')
print(f'Bought {len(vendors_nv_success)} asked vendors\' products:\n {vendors_nv_success}')

Task completed: False
Bought 0 asked vendors' products:
 set()


###***Diversity in category***

Task: Buy goods from one of the `proposed/recommended` categories.

Implementation: Recommend few categories.

In [29]:
from collections import Counter

def get_diversity_in_category_task(client_id, num_categories=2, num_required=1, num_examples=3, mode='all'):
    print(f'*Diversity in category task ({mode} products)*')
    mode_message = 'previously purchased' if mode == 'bought' else 'novel for you'
    print(f'Buy at least 4 different goods from one of the following hier_level_2 categories at a time:\n')

    categories = recommend_items(
        model_hier_2, user_item_hier_2, client_id=client_id, field='hier_level_2',
        N=num_categories, mode=mode, # 'all', 'bought', 'novel',
    )
    category_names = list(map(lambda x: x['item'], categories))
    for cat_name in category_names:
        print(f' - {cat_name}')

    return category_names

def check_diversity_in_category_task(client_id, categories_task, num_required_materials=4):
    check = get_last_check(client_id)
    categories_bought = check.material\
        .apply(lambda x: materials_map[x])\
        .apply(lambda x: materials_fields_maps['hier_level_2'][x])\
        .apply(lambda x: materials_fields_maps['hier_level_2_ids_inv'][x])
    categories_count = Counter(categories_bought)

    for cat_name in categories_task:
        if cat_name in categories_count and categories_count[cat_name] >= num_required_materials:
            return True, cat_name, categories_count[cat_name]
    return False, None, 0

In [30]:
num_categories = 2
num_required = 1
num_required_materials = 4

categories_dc = get_diversity_in_category_task(
    client_idx, num_categories=num_categories, num_required=num_required, 
    num_examples=3, mode='all'
)

*Diversity in category task (all products)*
Buy at least 4 different goods from one of the following hier_level_2 categories at a time:

 - 642e92efb79421734881b53e1e1b18b6
 - 67c6a1e7ce56d3d6fa748ab6d9af3fd7


Check task completion based on the last receipt:

In [31]:
success_dc, category_dc_success, num_dc_success = check_diversity_in_category_task(
    client_id, categories_dc, num_required_materials=num_required_materials
)
print(f'Task completed: {success_dc}')
if success_dc:
    print(f'Bought {num_dc_success} products in category {category_dc_success}')

Task completed: False


### ***Diverse Basket***

Goal: Enforce buying products from different categories

In [34]:
def get_diverse_basket_task(client_id, num_categories=5, num_required=3, num_examples=3, mode='bought'):
    print(f'*Diverse Basket task ({mode} products)*')
    mode_message = 'previously purchased' if mode == 'bought' else 'novel for you'
    print(f'Fill your cart with products from at least {num_required}/{num_categories} '
          f'{mode_message} hier_level_2 categories:')

    categories = recommend_items(
        model_hier_2, user_item_hier_2, client_id=client_id, field='hier_level_2',
        N=num_categories, mode=mode, # 'all', 'bought', 'novel',
    )

    category_names = list(map(lambda x: x['item'], categories))
    for cat_name in category_names:
        print(f'\nCategory {cat_name}\nExample products: ')

        if num_examples:
            cat_materials = recommend_items(
                model_material, user_item_material, client_id=client_id, field='material',
                N=num_examples, top_k=10000,
                mode='all', # 'all', 'bought', 'novel'
                hier_level_2=cat_name,
            )

            for i, m in enumerate(cat_materials):
                print(f' - {m["item"]["material"]}')

    return category_names
    

def check_diverse_basket_task(client_id, categories_task, num_required=3):
    check = get_last_check(client_id)
    categories_bought = check.material\
        .apply(lambda x: materials_map[x])\
        .apply(lambda x: materials_fields_maps['hier_level_2'][x])\
        .apply(lambda x: materials_fields_maps['hier_level_2_ids_inv'][x]).unique()
    categories_bought = set(categories_bought)
    categories_task = set(categories_task)
    categories_intersection = categories_task.intersection(categories_bought)
    return len(categories_intersection) >= num_required, categories_intersection

#### *Diverse Basket (known categories)*

Task: Fill your cart with products from at least 7/10 previously bought `hier_level_2` categories.

Implementation: Recommend 10 `hier_level_2` "bought" categories and optionally suggest few goods from each category:

In [37]:
num_categories = 10
num_required = 7
num_examples = 3

categories_dbk = get_diverse_basket_task(
    client_idx, num_categories=num_categories, num_required=num_required, 
    num_examples=num_examples, mode='bought'
)

*Diverse Basket task (bought products)*
Fill your cart with products from at least 7/10 previously purchased hier_level_2 categories:
10

Category 34173cb38f07f89ddbebc2ac9128303f
Example products: 
 - 9817112604594b376adf135efe787bbc
 - 19056d14d938838ccff7c13c48d37285
 - 96293d6eb5849a7fe116b8a184af58cb

Category fc490ca45c00b1249bbe3554a4fdf6fb
Example products: 
 - a870783c7818cc9a3b4a45f13e3036da
 - 35fb842aa9089ffd980a306df35bf533
 - c6a6fed3cabbe8efd41135033f9d0961

Category 02e74f10e0327ad868d138f2b4fdd6f0
Example products: 
 - e0f04658c07ce064eccc97ce6c0e3d7f
 - a8149429fb72f9518cdb560b3cf62425
 - f09b39028a61179e455fa9c072397a65

Category 4e732ced3463d06de0ca9a15b6153677
Example products: 
 - 376eec543ce52c6af676b423b9914c25
 - 4b54e5205b767664f161323eb2ceed20
 - 9422d8ab9ac239bdc841cd84e87e29ab

Category 70efdf2ec9b086079795c442636b55fb
Example products: 
 - ccf83ea1c9a46a646aebeeefee8ba014
 - dad9cb5d93d49b56e940e1a9d275f9fd
 - b4090db225b288f1ce5346424d9863b4

Category 8e2

Check task completion based on the last receipt:

In [38]:
success_dbk, categories_dbk_success = check_diverse_basket_task(
    client_id, categories_dbk, num_required=num_required
)
print(f'Task completed: {success_dbk}')
print(f'Bought in the following {len(categories_dbk_success)} categories:\n {categories_dbk_success}')

Task completed: False
Bought in the following 2 categories:
 {'70efdf2ec9b086079795c442636b55fb', '34173cb38f07f89ddbebc2ac9128303f'}


#### *Diverse Basket (novel categories)*

Task: Fill your cart with products from at least 1/2 novel `hier_level_2` categories.

Implementation: Recommend 2 `hier_level_2` novel categories and optionally suggest few goods from each category:

In [39]:
num_categories = 2
num_required = 1
num_examples = 3

categories_dbn = get_diverse_basket_task(
    client_idx, num_categories=num_categories, num_required=num_required, 
    num_examples=num_examples, mode='novel'
)

*Diverse Basket task (novel products)*
Fill your cart with products from at least 1/2 novel for you hier_level_2 categories:
2

Category 642e92efb79421734881b53e1e1b18b6
Example products: 
 - 564840544bef2c259cf7a0642ab97b2e
 - 662a483a37bda6993a3be40ad0f12d95
 - a68c3af6a344b97f58973b2a8a815e3a

Category 67c6a1e7ce56d3d6fa748ab6d9af3fd7
Example products: 
 - d217bef07f10f8e4610d71f509e6eb2a
 - 6d4a10434dcdd00fe8ac96487d1a2997
 - 2e995e794c8de54aae8facf99d5d1421


Check task completion based on the last receipt.

Note: no success since we suggest to buy products in novel categories

In [40]:
success_dbn, categories_dbn_success  = check_diverse_basket_task(
    client_id, categories_dbn, num_required=num_required
)
print(f'Task completed: {success_dbn}')
print(f'Bought in the following {len(categories_dbn_success)} categories:\n {categories_dbn_success}')

Task completed: False
Bought in the following 0 categories:
 set()


# Achievements

Example achievements in the proposed system with achievement validation functions:

###***Krusenstern***

Description: Visit more than 3 Lenta stores

In [41]:
def check_krusenstern(client_data, num_places=3):
    '''Visit more than `num_places` Lenta stores'''
    num_different_places = client_data['plant'].unique().shape[0]
    return num_different_places >= num_places

###***PartyGoer***

Description: Buy alcohol on more than 10000 rubles

In [42]:
def check_partygoer(client_data, check_sum=10000):
    '''Buy alcohol on more than `check_sum` rubles'''
    sales_sum = client_data[client_data['is_alco']==1]['sales_sum']
    return np.sum(sales_sum) > check_sum

###***Sweet Tooth***

Description: Buy sweets on >500 rubles

In [43]:
def check_sweet_tooth(client_data, min_sum=500):
    '''Buy sweets on >`min_sum` rubles'''
    sweet_id = df_materials['material'][1] # for example
    sales_sum = client_data[client_data['material']==sweet_id]['sales_sum']
    return np.sum(sales_sum) > min_sum

###***Greta Thunberg***

Description: Buy 10 reusable bags

In [44]:
def check_greta_thunberg(client_data, num_bags=10):
    '''Buy `num_bags` reusable bags'''
    reusable_bags_id = df_materials['material'][0] # for example
    sales_count = client_data[client_data['material']==reusable_bags_id]['sales_count']
    return np.sum(sales_count) >= num_bags

###***Milk King***

Description: Buy all brands of Milk

In [45]:
def check_milk_king(client_data):
    '''Buy all brands of Milk'''
    hier_level_3_id_milk = df_materials['hier_level_3'][0] # for example
    full_amount_of_milk_products = df_materials[df_materials['hier_level_3']==hier_level_3_id_milk]['hier_level_4'].unique()
    user_amount_of_milk_products = client_data[client_data['hier_level_3']==hier_level_3_id_milk]['hier_level_4'].unique()
    return set(full_amount_of_milk_products).issubset(user_amount_of_milk_products)

###***Lucky Devil***

Description: Use discounts 300 times

In [46]:
def check_lucky_devil(client_data, num_times=300):
    '''Use discounts `num_times` times'''
    promo_sum = client_data['is_promo'].sum()
    return promo_sum > num_times

###***Loyal Customer***

Description: Buy something in Lenta N=`num_weeks` weeks in a row

In [47]:
from itertools import groupby

def check_loyal_customer(client_data, num_weeks=4): 
    '''Buy something in Lenta N=`num_weeks` weeks in a row'''
    list_of_weeks = client_data['week']
    purchases_weeks = [el for el, _ in groupby(list_of_weeks)]
    count=0
    four_week_in_row = False
    for i in range(len(purchases_weeks)-1):
        if purchases_weeks[i]== purchases_weeks[i+1]-1:
            count+=1
        else:
            count = 0
        if count == num_weeks:
            four_week_in_row = True
            break
    return four_week_in_row

## Check achivements of an arbitrary client

In [48]:
achievement_funcs = {
    'Krusenstern': check_krusenstern,
    'PartyGoer': check_partygoer,
    'Sweet Tooth': check_sweet_tooth,
    'Greta Thunberg': check_greta_thunberg,
    'Milk King': check_milk_king,
    'Lucky Devil': check_lucky_devil,
    'Loyal Customer': check_loyal_customer
}

In [75]:
# specify a client
client_idx = 73281 #np.random.randint(0, len(clients_map))
client = df_clients.iloc[client_idx]
client_id = client.client_id
print(client)

client_id    7fe963af1319898ffd8c8fdaaed31be4
gender                                      F
city                                    Other
birthyear                                1957
Name: 73281, dtype: object


In [76]:
def check_achievements(client_id):
    client_achivements = {}
    client_data = df_transactions[df_transactions['client_id']==client_id]
    client_data = client_data.merge(df_materials)
    for name, func in achievement_funcs.items():
        client_achivements[name] = func(client_data)
    return client_achivements

client_achivements = check_achievements(client_id)
print('Client achievements:')
client_achivements

Client achievements:


{'Greta Thunberg': False,
 'Krusenstern': True,
 'Loyal Customer': True,
 'Lucky Devil': False,
 'Milk King': False,
 'PartyGoer': False,
 'Sweet Tooth': False}

# Model evalution

Small bits of model evaluation based on held-out transactions and MAP@K score.

In [114]:
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, train_test_split
from implicit.als import AlternatingLeastSquares
from implicit.datasets.movielens import get_movielens

`hier_level_2`: MAP@10

In [ ]:
train, test = train_test_split(user_item_hier_2.T, 0.8)

model = AlternatingLeastSquares(factors=32, regularization=0.1, iterations=50)
model.fit(train)

map_10 = mean_average_precision_at_k(model, train.T.tocsr(), test.T.tocsr(), K=10, num_threads=5)
map_10

0.4334339727530992

`material`: MAP@10

In [ ]:
train, test = train_test_split(user_item_material.T, 0.8)

model = AlternatingLeastSquares(factors=32, regularization=0.1, iterations=50)
model.fit(train)

map_10 = mean_average_precision_at_k(model, train.T.tocsr(), test.T.tocsr(), K=10, num_threads=)
map_10

0.09698819018992372